# Imports

In [36]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from scipy.fftpack import fft, dct
from scipy.stats import skew, kurtosis, entropy

# Load data

In [38]:
a = pd.read_pickle("data/vit/vit_a_correct.pkl")
b = pd.read_pickle("data/vit/vit_b_correct.pkl")
c = pd.read_pickle("data/vit/vit_c_correct.pkl")
d = pd.read_pickle("data/vit/vit_d_correct.pkl")
e = pd.read_pickle("data/vit/vit_e_correct.pkl")
video_correct = pd.concat([a,b,c,d,e], ignore_index=True)
print(video_correct.shape)
video_correct.to_pickle("data/vit/video_correct.pkl")

(26704, 31)


In [39]:
f = pd.read_pickle("data/vit/vit_a_incorrect.pkl")
g = pd.read_pickle("data/vit/vit_b_incorrect.pkl")
h = pd.read_pickle("data/vit/vit_c_incorrect.pkl")
i = pd.read_pickle("data/vit/vit_d_incorrect.pkl")
j = pd.read_pickle("data/vit/vit_e_incorrect.pkl")
video_incorrect = pd.concat([f,g,h,i,j], ignore_index=True)
print(video_incorrect.shape)
video_incorrect.to_pickle("data/vit/video_incorrect.pkl")

(21500, 31)


Annotate based on imu + scaling + position reset to 0

# Label sets and reps

In [40]:
video = pd.read_pickle("data/vit/video_correct.pkl") # Video data without annotations
imu = pd.read_pickle("data/out/df_correct_reps.pkl")  # IMU data with annotations
video = video.merge(imu[['Frame', 'Participant', 'Set', 'Repetition']], on=['Frame', 'Participant'], how='left')
video = video.dropna(subset=['Set', 'Repetition'])
video['Time'] = video['Frame'] / 60  

In [41]:
# For incorrect files
video2 = pd.read_pickle("data/vit/video_incorrect.pkl") 
imu2 = pd.read_pickle("data/out/df_incorrect_reps.pkl")
video2 = video2.merge(imu2[['Frame', 'Participant', 'Set', 'Repetition']], on=['Frame', 'Participant'], how='left')
video2 = video2.dropna(subset=['Set', 'Repetition'])
video2['Time'] = video2['Frame'] / 60 

# Rescale and reset positions

In [32]:
def reset_and_scale_positions(df):
    columns_to_process = [
        col for col in df.columns 
        if not any(keyword in col for keyword in ['acc', 'vel', 'Correct', 'Participant', 'Frame', 'Set', 'Repetition', 'Time'])
    ]
    df[columns_to_process] = df.groupby(['Set', 'Repetition'])[columns_to_process].transform(lambda x: x - x.iloc[0])
    scaler = MinMaxScaler()
    df[columns_to_process] = scaler.fit_transform(df[columns_to_process])
    return df

In [34]:
video_correct_done = reset_and_scale_positions(video) 
video_correct_done.to_pickle("data/out/video_correct_with_set_rep.pkl")
print("Annotated video data saved successfully.")

video_incorrect_done = reset_and_scale_positions(video2)
video_incorrect_done.to_pickle("data/out/video_incorrect_with_set_rep.pkl")
print("Annotated incorrect video data saved successfully.")

Annotated video data saved successfully.
Annotated incorrect video data saved successfully.
